In [1]:
import pandas as pd
import janitor as jn
import numpy as np
import numba as nb


In [2]:
url = "https://raw.githubusercontent.com/samukweku/data-wrangling-blog/master/notebooks/Data_files/flights.csv"
flights = pd.read_csv(url, sep = '\t', names=['orig','dest','orig_time', 'dest_time'], parse_dates  = ['orig_time', 'dest_time'])
flights = flights.factorize_columns(['orig','dest']).iloc[:, 2:]
flights.columns = [ent.split("_")[0] if ent.endswith('enc') else ent for ent in flights]
flights.columns = ['takeoff','landing','orig','dest']
flights = flights.assign(start=flights.landing+pd.Timedelta(minutes=45), end=flights.landing+pd.Timedelta(hours=3))
flights.head()

,takeoff,landing,orig,dest,start,end
0,2021-11-27 07:15:00,2021-11-27 08:55:00,0,0,2021-11-27 09:40:00,2021-11-27 11:55:00
1,2021-11-27 20:05:00,2021-11-28 00:50:00,1,1,2021-11-28 01:35:00,2021-11-28 03:50:00
2,2021-11-27 21:00:00,2021-11-27 21:35:00,2,2,2021-11-27 22:20:00,2021-11-28 00:35:00
3,2021-11-27 21:15:00,2021-11-27 22:25:00,0,2,2021-11-27 23:10:00,2021-11-28 01:25:00
4,2021-11-26 11:40:00,2021-11-26 14:45:00,3,3,2021-11-26 15:30:00,2021-11-26 17:45:00


In [3]:
%%timeit
(flights
 .conditional_join(
     flights, 
     ('end', 'takeoff' ,'>='), 
    ('start', 'takeoff', '<='), 
     ('orig','orig','>='), 
    #  ('dest', 'orig', '=='), 
     df_columns = ['start', 'end', 'dest'],
     right_columns = ['takeoff', 'orig'],
     use_numba=False)
)

305 ms ± 11.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
(flights
 .conditional_join(
     flights, 
     ('end', 'takeoff' ,'>='), 
    ('start', 'takeoff', '<='), 
      ('orig','orig','>='), 
    #  ('dest', 'orig', '=='), 
     df_columns = ['start', 'end', 'dest'],
     right_columns = ['takeoff', 'orig'],
     use_numba=True)
)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


163 ms ± 911 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
# nb.config.DISABLE_JIT = True

In [6]:
# url = 'https://raw.githubusercontent.com/samukweku/data-wrangling-blog/master/notebooks/Data_files/results.csv'
# events = pd.read_csv(url, parse_dates=['start', 'end']).iloc[:, 1:]
# events.head()

In [7]:
# %%timeit
# (events
# .conditional_join(
#     events,
#     ('end', 'start', '>='),
#     ('start', 'end', '<='),
#     # ('id', 'id', '!='),
#     use_numba = False,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

In [8]:
# %%timeit
# (events
# .conditional_join(
#     events,
#     ('end', 'start', '>='),
#     ('start', 'end', '<='),
#     # ('id', 'id', '!='),
#     use_numba = True,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

In [9]:
# (events
# .conditional_join(
#     events,
#     ('end', 'start', '>='),
#     ('start', 'end', '<='),
#     # ('id', 'id', '!='),
#     use_numba = True,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

In [10]:
# (events
# .conditional_join(
#     events,
#     ('start', 'end', '<='),
#     ('end', 'start', '>='),
#     ('id', 'id', '!='),
#     use_numba = True,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

In [11]:
# %%timeit
# (events
# .conditional_join(
#     events,
#     ('start', 'end', '<='),
#     ('end', 'start', '>='),
#     ('id', 'id', '!='),
#     use_numba = True,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

In [12]:
# %%timeit
# (events
# .conditional_join(
#     events,
#     ('start', 'end', '<='),
#     ('end', 'start', '>='),
#     ('id', 'id', '!='),
#     use_numba = False,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

In [13]:

C=dict(key=[f"c{num}" for num in range(1,8)],
       vol=[35,15,5,35,18,90,17],
       profit=[45,35,55,12,15,55,11],
       unitsSold=[15,10,30,10,15,80,2]
       )
C=pd.DataFrame(C)


D=dict(key=[f'd{num}' for num in range(1,9)],
       vol=[20,50,15,16,40,20,40,2],
       profit=[30,10,12,52,35,20,30,57],
       unitsSold=[20,35,10,12,40,30,5,15]
       )
D=pd.DataFrame(D)

In [14]:
C

,key,vol,profit,unitsSold
0,c1,35,45,15
1,c2,15,35,10
2,c3,5,55,30
3,c4,35,12,10
4,c5,18,15,15
5,c6,90,55,80
6,c7,17,11,2


In [52]:
D.sort_values('vol')

,key,vol,profit,unitsSold
7,d8,2,57,15
2,d3,15,12,10
3,d4,16,52,12
0,d1,20,30,20
5,d6,20,20,30
4,d5,40,35,40
6,d7,40,30,5
1,d2,50,10,35


In [16]:
# C = pd.concat([C]*10_000)
# D = pd.concat([D]*1_00)

In [17]:
# C.conditional_join(D, ('vol','vol','<'),('profit','profit','>'),use_numba=True,keep='last')

In [18]:
# C.conditional_join(D, ('vol','vol','<'),('profit','profit','>'),use_numba=False,keep='last')

In [19]:
# D

In [20]:
# %timeit C.conditional_join(D, ('vol','vol','<'),('profit','profit','>'),use_numba=True,keep='last')

In [21]:
# C.conditional_join(D, ('profit','profit','>'),('vol','vol','<'),use_numba=False,keep='first')

In [22]:
# C.conditional
# 
# _join(D, ('profit','profit','>'),('vol','vol','<'),use_numba=True, keep='first')

In [23]:
# C.conditional_join(D, ('profit','profit','>'), ('vol','vol','<'),('unitsSold','unitsSold','>'),use_numba=True,keep='first')

In [24]:
# C.conditional_join(D, ('profit','profit','>'), ('vol','vol','<'),('unitsSold','unitsSold','>'),use_numba=False,keep='last')

In [25]:
# C.conditional_join(D, ('vol','vol','<'),('profit','profit','>'), ('unitsSold','unitsSold','>'),use_numba=False,keep='all')

In [26]:
# %timeit C.conditional_join(D, ('vol','vol','<'),('profit','profit','>'), ('unitsSold','unitsSold','>'),use_numba=True,keep='all')

In [27]:
%timeit C.conditional_join(D, ('vol','vol','<'),('profit','profit','>'), ('unitsSold','unitsSold','>'),use_numba=False,keep='all')

1.31 ms ± 27.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [28]:
# C.conditional_join(D, ('vol','vol','<'),('profit','profit','>'),use_numba=False, keep='last')

In [29]:
df1 = pd.DataFrame({"id": [1, 1, 1, 2, 2, 3], "value_1": [2, 5, 7, 1, 23, 4]})
df2 = pd.DataFrame(
    {
        "id": [1, 1, 1, 1, 2, 2, 2, 3],
        "value_2A": [0, 3, 7, 12, 0, 2, 3, 1],
        "value_2B": [1, 5, 9, 15, 1, 4, 6, 3],
    }
)
df1 = df1.astype({'value_1':'Int64'})
df2 = df2.astype({'value_2A':'Int64', 'value_2B': 'Int64'})
# df2=df2.sort_values('value_2A', ignore_index=True)

In [30]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,23
5,3,4


In [31]:
df2.sort_values('value_2B')

,id,value_2A,value_2B
0,1,0,1
4,2,0,1
7,3,1,3
5,2,2,4
1,1,3,5
6,2,3,6
2,1,7,9
3,1,12,15


In [32]:
# df1 = pd.concat([df1]*10_000)
# df2 = pd.concat([df2]*200)

In [33]:

df1.conditional_join(
    df2,
      ("value_1", "value_2B", "<="),
      ("value_1", "value_2A", ">="),
    #  ('id', 'id', '=='),
    use_numba=False,
    how="inner",
    indicator=False
)

left         right                  
     id value_1    id value_2A value_2B
0     1       2     3        1        3
1     1       2     2        2        4
2     1       5     1        3        5
3     1       5     2        3        6
4     1       7     1        7        9
5     2       1     1        0        1
6     2       1     2        0        1
7     2       1     3        1        3
8     3       4     2        2        4
9     3       4     1        3        5
10    3       4     2        3        6

In [34]:
df1.conditional_join(
    df2,
      ("value_1", "value_2B", "<="),
      ("value_1", "value_2A", ">="),
    #  ('id', 'id', '=='),
    use_numba=True,
    how="inner",
    indicator=False
)

left         right                  
     id value_1    id value_2A value_2B
0     2       1     1        0        1
1     2       1     2        0        1
2     2       1     3        1        3
3     1       2     3        1        3
4     1       2     2        2        4
5     3       4     2        2        4
6     3       4     1        3        5
7     3       4     2        3        6
8     1       5     1        3        5
9     1       5     2        3        6
10    1       7     1        7        9

In [35]:
# %%timeit
# df1.conditional_join(
#     df2,
#       ("value_1", "value_2B", "<="),
#       ("value_1", "value_2A", ">="),
#       ('id', 'id', '=='),
#     use_numba=True,
#     how="left",
#     indicator=False
# )

In [36]:
# %%timeit
# df1.conditional_join(
#     df2,
#       ("value_1", "value_2B", "<="),
#       ("value_1", "value_2A", ">="),
#      ('id', 'id', '=='),
#     use_numba=False,
#     how="right",
#     indicator=True
# )

In [37]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,5
2,1,7,9
3,1,12,15
4,2,0,1
5,2,2,4
6,2,3,6
7,3,1,3


In [38]:
# # %%timeit
# df1.conditional_join(
#     df2,
#     #  ("value_1", "value_2B", "<="),
#       ("value_1", "value_2A", ">="),
#      ('id', 'id', '=='),
#     use_numba=True,
#     keep="all",
# )

In [39]:
# df1.conditional_join(
#     df2,
#     # ("value_1", "value_2B", "<"),
#      ("value_1", "value_2A", ">"),
#     # ('id', 'id', '>'),
#     use_numba=False,
#     keep="all",
# )

In [40]:
# df1.conditional_join(
#     df2,
#     ("value_1", "value_2B", "<"),
#     ("value_1", "value_2A", ">"),
#     # ('id', 'id', '>'),
#     use_numba=True,
#     keep="all",
# )

In [41]:
# df1.conditional_join(
#     df2,
#         ("value_1", "value_2B", ">"),
#      ("value_1", "value_2A", ">"),

   
#     use_numba=True,
#     keep="first",
# )

In [42]:
# df1.conditional_join(
#     df2,
#      ("value_1", "value_2A", "<"),
#     ("value_1", "value_2B", "<"),
   
#     use_numba=False,
#     keep="all",
# )

In [43]:
# df1.conditional_join(
#     df2,
#      ("value_1", "value_2A", ">"),
#     ("value_1", "value_2B", "<"),
   
#     use_numba=False,
#     keep="last",
# )

In [44]:
# df1.conditional_join(
#     df2,
#     ("value_1", "value_2B", "<"),
#     ("value_1", "value_2A", ">"),
#     use_numba=True,
#     keep="all",
# )

In [45]:
# df1.conditional_join(
#     df2,
#     ("value_1", "value_2A", ">"),
#     ("value_1", "value_2B", "<"),
#     use_numba=True,
#     keep="last",
# )

In [46]:
# df1.conditional_join(df2, ('value_1', 'value_2A','>'), ('value_1','value_2B', '<'), use_numba=False,keep='all')

In [47]:
from math import inf
df = {#'A':[0],'B':[0.0],
    # "B": [ 2.836912e+14,1.100000e+00,-inf,1.100000e+00,-1.643857e-193         ],
     "E": [pd.NaT],
}
df = pd.DataFrame(df)
df

,E
0,NaT


In [48]:
right = {
    # "Floats": [0.0], "Integers":[0],
    #  "Floats": [-1.000000e+00,-1.000000e+00, -1.192093e-07,-1.000000e+00,-1.000000e+00,1.980469e+00],
    # "Strings": [""]*2,
    # "Booleans": [False]*2,
    "Dates": pd.to_datetime([pd.NaT]),
    # "Dates_Right": pd.to_datetime(['1970-01-01 00:00:00.000000000','1970-01-01 00:00:00.000000001']),
}
right = pd.DataFrame(right)
right

,Dates
0,NaT


In [49]:
jn.get_join_indices(
        df.dropna(subset=["E"]),
        right.dropna(subset=["Dates"]),
        [
            ("E", "Dates", "=="),
        ],
        return_ragged_arrays=True,
    )

(array([], dtype=int64), array([], dtype=int64))

In [50]:
actual = (
        df[["B"]]
        .conditional_join(
            right[["Numeric", "Floats"]],
            ("B", "Floats", "<="),
            ("B", "Numeric", ">="),
            how="inner",
            use_numba=True,
        )
        # .sort_values(["B", "Numeric", "Floats"], ignore_index=True)
    )

actual

KeyError: "None of [Index(['B'], dtype='object')] are in the [columns]"

In [47]:


# actual = (
#     df[["B", "E"]]
#     .conditional_join(
#         right[["Numeric", "Dates"]],
#         ("E", "Dates", ">="),
#             ("B", "Numeric", "<"),
#             how="inner",
#             use_numba=True,
#         )
#     )
# actual

In [48]:
# filters = ["A", "B", "Integers", "Numeric"]

# expected = df[["A", "B"]].merge(
#     right[["Integers", "Numeric"]], how="cross"
# )
# expected = expected.loc[
#         expected.A.ne(expected.Integers) & expected.B.ne(expected.Numeric)
#     ].sort_values(filters, ignore_index=True)

# expected

In [49]:
# actual = (
#         df[["A", "B"]]
#         .conditional_join(
#             right[["Integers", "Numeric"]],
#             ("A", "Integers", "!="),
#             ("B", "Numeric", "!="),
#             how="inner",
#         )
#         # .sort_values(filters, ignore_index=True)
#     )


# actual

In [50]:
# # df = df.sort_values("B").dropna(subset=["B"])
# actual = (
#         df[["A"]]
#         .conditional_join(
#             right[["Integers"]],
#             ("A", "Integers", "<"),
#             how="inner",
#             # keep="first",
#             use_numba=True,
#         )
#         # .sort_values(["A", "Integers"], ignore_index=True)
#     )

# actual

In [51]:
# (
#         df[["A"]]
#         .conditional_join(
#             right[["Integers"]],
#             ("A", "Integers", "<"),
#             how="inner",
#             # keep="first",
#             use_numba=False,
#         )
#      #   .sort_values(["B", "Numeric"], ignore_index=True)
#     )

In [52]:
# (
#         df[["A"]]
#         .merge(right[["Integers"]], how="cross")
#         .loc[lambda df: df.A.lt(df.Integers)]
#         .sort_values(["A", "Integers"], ignore_index=True)
#     )